# 3. Exploratory Data Analysis

## 3.1 Importing Modules

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns


## 3.2 Importing Data

In [2]:
movies_metadata=pd.read_csv('../data/movies_metadata_cleaned.csv')
ratings=pd.read_csv('../data/ratings.cleaned.csv')
keywords=pd.read_csv('../data/keywords_cleaned.csv')
links=pd.read_csv('../data/links_cleaned.csv')

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0,3,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
